## Connection

In [1]:
import os

from dotenv import load_dotenv
import numpy as np
import pandas as pd
from sqlalchemy import create_engine


# creds from .env file
load_dotenv()

POSTGRES_HOST = os.getenv('POSTGRES_HOST')
POSTGRES_PORT = os.getenv('POSTGRES_PORT')
POSTGRES_USERNAME = os.getenv('POSTGRES_USERNAME')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
POSTGRES_DBNAME = os.getenv('POSTGRES_DBNAME')


# connection
try:
    conn = create_engine(f"postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DBNAME}")
    print('connected!')
except:
    print('Can`t establish connection to database')


# small function for selecting data
def select(sql):
    return pd.read_sql_query(sql, conn)

connected!


In [2]:
sql = ''' 
SELECT *
FROM products
'''
select(sql)


,id,name,price
0,2,Monthly subscription,14.99
1,3,Yealry subscription,89.99


In [3]:
sql = ''' 
SELECT *
FROM purchases
'''
select(sql)


,id,user_id,product_id,amount,refunded,created_at
0,1,2306,2,14.99,False,2018-11-25 23:29:23.063394
1,2,2316,2,14.99,False,2018-05-17 21:35:18.482118
2,3,2355,2,14.99,False,2018-10-21 21:09:35.543070
3,4,2374,2,14.99,False,2018-09-23 01:11:33.904443
4,5,2416,2,14.99,False,2018-12-04 18:01:14.317714
...,...,...,...,...,...,...
911,912,5407,2,14.99,False,2018-02-09 08:44:00.662207
912,913,5410,2,14.99,False,2018-02-09 07:47:05.437012
913,914,5424,3,89.99,False,2018-02-20 00:03:02.296503
914,915,5425,3,89.99,False,2018-02-21 14:22:58.711632


In [21]:
sql = ''' 
select
    to_char(created_at, 'yyyy-mm'),  sum(amount)
from purchases 
where refunded = FALSE
group by 1
order by 1 desc

'''
select(sql)

,to_char,sum
0,2019-01,416.90997
1,2018-12,1691.68980
2,2018-11,1463.71980
3,2018-10,737.81990
4,2018-09,1019.75980
5,2018-08,1499.72990
6,2018-07,1598.70980
7,2018-06,1265.77990
8,2018-05,1733.66980
9,2018-04,2162.58980


In [4]:
sql = ''' 
SELECT
  CASE 
    WHEN product_id = 2 THEN 'monthly'
    WHEN product_id = 3 THEN 'yearly'
    ELSE 'other_plan'
  END AS plan,
  SUM(amount) AS revenue
FROM purchases
WHERE
  refunded = FALSE
GROUP BY 1
'''
select(sql)


,plan,revenue
0,monthly,4817.6587
1,yearly,40909.1520


In [31]:
sql = ''' 
select * from purchases AS p
inner join users AS u on u.id = p.user_id
'''
select(sql)



,id,user_id,product_id,amount,refunded,created_at,id,email,first_name,last_name,...,utm_campaign,utm_medium,utm_term,utm_content,visitor_id,status,adjust_tracker,adjust_campaign,adjust_adgroup,adjust_creative
0,1,2306,2,14.99,False,2018-11-25 23:29:23.063394,2306,van@moore.com,Waylon,Ratke,...,,,,,10f6c4eb6a5c23a8,free,,,,
1,2,2316,2,14.99,False,2018-05-17 21:35:18.482118,2316,florenciokohler@johnson.net,Armando,Runolfsdottir,...,,,,,33c8ebcb75f6ee24,free,,,,
2,3,2355,2,14.99,False,2018-10-21 21:09:35.543070,2355,mae@pfefferblanda.info,Mohammad,Kiehn,...,,,,,3995f6509945e230,trial,,,,
3,4,2374,2,14.99,False,2018-09-23 01:11:33.904443,2374,kasey@heel.co,Alejandro,Hayes,...,,,,,c6b51932e4bb7fe3,trial,,,,
4,5,2416,2,14.99,False,2018-12-04 18:01:14.317714,2416,earlehuels@rolfsondouglas.net,Johnnie,Morissette,...,,,,,0933d4cfcfc45bf1,trial,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,912,5407,2,14.99,False,2018-02-09 08:44:00.662207,5407,marge@treutel.name,Bee,Grant,...,,,,,5ad605bf937a277a,customer,,,,
912,913,5410,2,14.99,False,2018-02-09 07:47:05.437012,5410,nicholas@muller.net,Seth,Bailey,...,,,,,e91df5a175c90d4c,customer,,,,
913,914,5424,3,89.99,False,2018-02-20 00:03:02.296503,5424,corey@harber.biz,Vincenzo,Jaskolski,...,,,,,a622efe1e09c48e8,customer,,,,
914,915,5425,3,89.99,False,2018-02-21 14:22:58.711632,5425,arden@funk.org,Hank,Balistreri,...,,,,,94912eb045ba0025,customer,,,,


In [33]:
sql = ''' 
SELECT
  pr.name,
  SUM(amount) AS revenue
FROM purchases pu
INNER JOIN products pr
  ON pu.product_id = pr.id
WHERE
  refunded = FALSE
GROUP BY 1

'''
select(sql)

,name,revenue
0,Yealry subscription,40909.1520
1,Monthly subscription,4817.6587


In [42]:
sql = ''' 
select 
    book_id
    , b.name
    , count(user_id)
from books_users bu
    left join books b on b.id = bu.book_id
group by 1, 2
order by 3 desc, 2 asc
limit 1
'''
select(sql)



,book_id,name,count
0,282,Snake of Rainbow,7


In [44]:
sql = '''
WITH campaign_revenues AS (
  SELECT
    utm_source,
    SUM(amount) AS total_revenue
  FROM purchases p
  INNER JOIN users u
    ON p.user_id = u.id
  WHERE
    refunded = FALSE
    AND utm_source IS NOT NULL	  
  GROUP BY 1
), campaign_spends AS (
  SELECT
    utm_source,
    SUM(amount) AS total_spend
  FROM marketing_spends
  GROUP BY 1
)

SELECT 
  s.utm_source,
  total_spend,
  total_revenue,
  (total_revenue - total_spend) / total_spend AS ROI
FROM campaign_spends s
LEFT JOIN campaign_revenues r
  ON s.utm_source = r.utm_source

'''
select(sql)

,utm_source,total_spend,total_revenue,roi
0,facebook,584.8600,1328.7599,1.271928
1,twitter,2920.6702,6499.9090,1.225486
2,gdn,629.5100,NaN,NaN
